In [1]:
import numpy as np
import pandas as pd
import requests
import json
import time

In [2]:
import os
from TabularExperimentTrackerClient.ExperimentClient import ExperimentClient

path =  '../../creds/'
creds_orch_file = "creds-orch.txt"

with open(os.path.join(path, creds_orch_file), "r") as myfile:
    usr_name = myfile.readline().strip()  # Read the first line and strip any leading/trailing whitespace
    tabExpTrackAPIKey = myfile.readline().strip()  # Read the second line and strip any leading/trailing whitespace

print('API Key Loaded!')

API Key Loaded!


In [5]:
ex = ExperimentClient(verbose = True)
ex.define_orch_cred(usr_name, tabExpTrackAPIKey)

In [7]:
from requests.exceptions import RequestsDependencyWarning

expname = "Mlp_experiment_1"

slice_start = 0
slice_size = 1000
i = 0
data = []
while True:
    url = ""
    payload = json.dumps({"experiment": expname, "slice_start": slice_start, "slice_end": slice_start+slice_size})
    headers = {'Name': usr_name,'Seceret': tabExpTrackAPIKey,'Content-Type': 'application/json'}
    t1 = time.time()
    resp_unparse = requests.request("POST", url, headers=headers, data=payload)
    t2=time.time()
    resp = json.loads(resp_unparse.text)
    t3=time.time()
    print('========= {} ========'.format(i))
    print('request time: {}'.format(t2-t1))
    print('parse time: {}'.format(t3-t2))
    print('length: {}'.format(len(resp)))
    data.extend(resp)
    if len(resp) == 0:
        break
    slice_start+=slice_size
    i+=1

print('total length: {}'.format(len(data)))

========= 0 ========
request time: 5.203074216842651
parse time: 0.14678382873535156
length: 1000
========= 1 ========
request time: 3.8291521072387695
parse time: 0.0687568187713623
length: 1000
========= 2 ========
request time: 6.802064895629883
parse time: 0.07110404968261719
length: 1000
========= 3 ========
request time: 3.555927038192749
parse time: 0.0392611026763916
length: 1000
========= 4 ========
request time: 3.6083290576934814
parse time: 0.06936407089233398
length: 1000
========= 5 ========
request time: 3.577115058898926
parse time: 0.0695958137512207
length: 1000
========= 6 ========
request time: 4.973895072937012
parse time: 0.13472223281860352
length: 1000
========= 7 ========
request time: 2.998137950897217
parse time: 0.01810622215270996
length: 105


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [9]:
len(data)

7105

In [10]:
df_model = [] # model
df_dataset = [] # task
df_hyperparameters = [] # hyp
df_metrics = []


for i, observation in enumerate(data):
    df_model.append(observation['model'])
    df_dataset.append(observation['task'])
    df_hyperparameters.append(observation['hyp'])
    try:
        df_metrics.append(observation['metrics_per_epoch'][-1])
    except:
        df_metrics.append(None)
else:
    mlp_df = pd.DataFrame({
        'model': df_model,
        'dataset': df_dataset,
        'hyperparameters': df_hyperparameters,
        'metrics': df_metrics
    })

In [11]:
print(mlp_df)

              model     dataset   
0     Mlp_sigm_cont  337-361275  \
1      Mlp_relu_cat  334-361110   
2     Mlp_sigm_cont  336-361074   
3      Mlp_sigm_cat  335-361104   
4      Mlp_relu_cat  335-361097   
...             ...         ...   
7100  Mlp_relu_cont  336-361279   
7101   Mlp_relu_cat  335-361289   
7102  Mlp_relu_cont  336-361279   
7103   Mlp_relu_cat  335-361289   
7104   Mlp_relu_cat  335-361289   

                                        hyperparameters   
0     {'depth': 12, 'seed': 42, 'regularize': None, ...  \
1     {'depth': 9, 'seed': 42, 'regularize': 0.25, '...   
2     {'depth': 9, 'seed': 42, 'regularize': None, '...   
3     {'depth': 8, 'seed': 42, 'regularize': None, '...   
4     {'depth': 6, 'seed': 42, 'regularize': 'bn', '...   
...                                                 ...   
7100  {'depth': 2, 'seed': 42, 'regularize': 0.5, 'e...   
7101  {'depth': 2, 'seed': 42, 'regularize': None, '...   
7102  {'depth': 10, 'seed': 42, 'regularize': No

In [12]:
mlp_df.to_csv('MlpTotalResults.csv', index=False)